# Задача 1

---

## Задание

Дана матрица А осцилляционного типа. Построим СЛАУ $Az = u$ в которой правая часть u вычисляется по формуле

$u = Az_0, где z_0 = (1, 1, …, 1)$

Строим матрицу B = sqrt(A) (см. методическое пособие стр. 22). 

Убедитесь, что матрица А осцилляционного типа. 

Заполняет таблицу 

$n=2     cond(A)=…     cond(B)=…    || A - B^2 ||=…$

$n=3     cond(A)=…     cond(B)=…    || A - B^2 ||=…$

до тех, пока $|| A - B^2 ||=…$ пренебрежимо малая величина. 

Далее для решения уравнения  применяем 2 способа регуляризации: 

    1) (A^* A + alpha E) z = A^* u          (уравнение (16) пособия) 

    2) (B^* B + alpha E) z = B^* (B^(-1) u) (уравнение (20) пособия) 

Провести сравнение полученных результатов и сделать заключение о рациональном выборе метода регуляризации и параметра регуляризации.

## Вариант 26.
В обобщенной матрице Вандермонда $b_k = (n + 1 - k) ^{(-3)}, a_k = k / 2, k = 1, …, n$

---


## Решение

In [56]:
import numpy as np
import pandas as pd

from scipy import linalg as la

from typing import Callable, Iterable

Вспомогательные функции для небольших вычислений:
- Вычисление чисел обусловленности
- Генерация обобщенной матрицы Вандермонда
- Генерация правой части уравнения

In [57]:
def cond(matrix: np.ndarray):
    return np.linalg.cond(matrix)

def generalized_vandermond(n: int):
    matrix = []

    a_k = np.array([(k + 1) / 2 for k in range(n)])
    for j in range(n):
        matrix.append(a_k ** (n + 1 - (j + 1)) ** (-3))

    return np.matrix(np.column_stack(matrix))

def get_right_side(matrix: np.ndarray):
    n, _ = matrix.shape
    return matrix @ np.ones( (n, 1) )


### Метод

- Строим осцилляционные матрицы размера n, как обобщенные матрицы вандермонда

- После чего строим матрицу $B = sqrt(A)$.

- И смотрим на зависимость по таблице.

- Далее решаем СЛАУ двумя методами регуляризации.

In [58]:
def fill_table(n: int, tolerance: float =10e-14, max_iter: int =100):
    df = pd.DataFrame()

    while n < max_iter:
        A = generalized_vandermond(n)
        B = la.sqrtm(A) # build-in function for the matrix we need
        
        error = la.norm(A - B @ B)
        if error > tolerance:
            break
        
        series = pd.Series(data=[n, cond(A), cond(B), error], index=['n', "cond(A)", "cond(B)", "error"])
        df = pd.concat([df, series], ignore_index=True, axis=1)
        n += 1

    # formatting the dataframe
    df = df.T.astype({"n": int})

    return df


result = fill_table(2, max_iter=100)
result

,n,cond(A),cond(B),error
0,2,7.274687e+00,2.721875e+00,6.661338e-16
1,3,4.557577e+02,2.202434e+01,1.316726e-14
2,4,1.743875e+05,5.020722e+02,5.474255e-15
3,5,2.167641e+08,2.196942e+04,2.548415e-14
4,6,6.617269e+11,1.546214e+06,4.827875e-15
5,7,4.118889e+15,1.703678e+08,2.037373e-14
6,8,8.646993e+16,1.044881e+09,3.939211e-14
7,9,3.014779e+18,1.190666e+09,5.190480e-14
8,10,1.508478e+18,1.213108e+09,3.097941e-14
9,11,4.658897e+18,3.393501e+09,1.239777e-14


In [59]:
def regularization_with_simple_matrix(matrix: np.ndarray, right_vect: np.ndarray, alpha: float):
    """
    Регуляризация первым методом
    """
    n, _ = matrix.shape
    ident = np.identity(n)

    left_side = matrix.H @ matrix + alpha * ident
    right_side = matrix.H @ right_vect

    return la.solve(left_side, right_side)
        
def regularization_with_simmetric_positive_defined_matrix(matrix: np.ndarray, right_vect: np.ndarray, alpha: float):
    """
    Регуляризация вторым методом
    """
    n, _ = matrix.shape
    ident = np.identity(n)

    matrix = np.matrix(la.sqrtm(matrix))

    left_side = matrix.H @ matrix + alpha * ident
    right_side = matrix.H @ (la.inv(matrix) @ right_vect)

    return la.solve(left_side, right_side)


METHOD_NAMES = {
    regularization_with_simple_matrix: "Способ 1",
    regularization_with_simmetric_positive_defined_matrix: "Способ 2"
}

### Эксперимент

Решаем СЛАУ сначала одним методом, потом другим. После чего сравниваем результаты.

In [60]:
def experiment(n: int, method: Callable, alpha_degree_range: Iterable):
    df = pd.DataFrame()
    matrix = generalized_vandermond(n)
    right_side = get_right_side(matrix)

    for alpha_degree in alpha_degree_range:
        alpha = 10 ** (alpha_degree)

        result = method(matrix, right_side, alpha)
        error = la.norm(result - np.ones(n))
        
        series = pd.Series(data=[n, alpha, error], index=['n', "alpha", f"error ({METHOD_NAMES[method]})"])
        df = pd.concat([df, series], ignore_index=True, axis=1)

    df = df.T.astype({"n": int})
    return df


Выбираем $n = 28$

In [61]:
n = 28
alphas = range(-10, 1)

result_first_method = experiment(n, regularization_with_simple_matrix, alphas)
result_second_method = experiment(n, regularization_with_simmetric_positive_defined_matrix, alphas)

result_first_method.merge(result_second_method, how="inner")

,n,alpha,error (Способ 1),error (Способ 2)
0,28,1.000000e-10,0.033890,1.581859
1,28,1.000000e-09,0.003570,0.176049
2,28,1.000000e-08,0.000353,0.037243
3,28,1.000000e-07,0.000054,0.031984
4,28,1.000000e-06,0.000324,0.024805
5,28,1.000000e-05,0.001170,0.008735
6,28,1.000000e-04,0.001777,0.003317
7,28,1.000000e-03,0.008185,0.018811
8,28,1.000000e-02,0.067303,0.054714
9,28,1.000000e-01,0.258055,0.272183


## Выводы

Наименьшая погрешность первым методом достигается при $alpha = 1.000000e-07$.
Наименьшая погрешность вторым методом достигается при $alpha = 1.000000e-04$.

При $alpha = 1$ погрешность решений начинает существенно увеличиваться, при
решении вторым способом она превышает единицу.

Та же самая зависимость наблюдается при выборе сильно малых $alpha$.
Так, при решении задачи вторым способом при $alpha = 1.000000e-10$ погрешность
превысила единицу.

В целом, первый метод регуляризации показывает большую точность, а значит лучше
применим в данной случае.

